In [77]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

In [78]:
# Import wine regions csv file as DataFrame
wine_region_pd = pd.read_csv("wine_regions.csv")
wine_region_pd.head()

,Wine Region,State or Country
0,Bordeaux,France
1,Saint Emilion,France
2,Beaujolais,France
3,Fleurie,France
4,Rhone Valley,France


In [79]:
# Add columns for lat, lng 
# Note that we used "" to specify initial entry.
wine_region_pd["Lat"] = ""
wine_region_pd["Lng"] = ""

wine_region_pd.head()

,Wine Region,State or Country,Lat,Lng
0,Bordeaux,France,,
1,Saint Emilion,France,,
2,Beaujolais,France,,
3,Fleurie,France,,
4,Rhone Valley,France,,


In [85]:
# Create a parameters dictionary that will be updated with new city / wine region each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city / wine region
for index, row in wine_region_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['Wine Region']
    state = row['State or Country']

    # Update address key value
    params['address'] = f"{city},{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    wine_region_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    wine_region_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
wine_region_pd.head()

,Wine Region,State or Country,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
0,Bordeaux,France,44.8378,-0.57918,Hilton Garden Inn Bordeaux Centre,"17 Allée de RIO, Bordeaux",4.3
1,Saint Emilion,France,44.8944,-0.155729,Chateau Hotel & Spa Grand Barrail,"ROUTE DE LIBOURNE, D243, Saint-Émilion",4.5
2,Beaujolais,France,46.1134,4.72639,Château d'Igé,"252 Rue du Château, Igé",4.4
3,Fleurie,France,46.1926,4.69746,B&B HOTEL,"43 Rue Raulin, Lyon",4.1
4,Rhone Valley,France,45.6403,5.6865,Château de Chapeau cornu,"312 Rue de la Garenne, Vignieu",4.5


In [86]:
# Parameters dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in wine_region_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip 
    # any that are missing a data point.
    try:
        wine_region_pd.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        wine_region_pd.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        wine_region_pd.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [88]:
# Save Data to csv
wine_region_pd.to_csv("Time_To_Drink_Wine.csv")

# Visualize to confirm airport data appears
wine_region_pd

,Wine Region,State or Country,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
0,Bordeaux,France,44.8378,-0.57918,Château Grattequina,"50 Avenue de Labarde, Blanquefort",4.6
1,Saint Emilion,France,44.8944,-0.155729,Chateau Hotel & Spa Grand Barrail,"ROUTE DE LIBOURNE, D243, Saint-Émilion",4.5
2,Beaujolais,France,46.1134,4.72639,Château d'Igé,"252 Rue du Château, Igé",4.4
3,Fleurie,France,46.1926,4.69746,Château d'Igé,"252 Rue du Château, Igé",4.4
4,Rhone Valley,France,45.6403,5.6865,Château de Candie,"533 Rue du Bois de Candie, Chambéry",4.4
5,Chateauneuf du Pape,France,44.0566,4.83128,Hotel De L'horloge Avignon,"1 Rue Félicien David, Avignon",4.2
6,Alsace,France,48.3182,7.44162,Hôtel Le Maréchal,"5 pl, Place des 6 Montagnes Noires, Colmar",4.5
7,Colmar,France,48.0794,7.35851,Chateau d'Isenbourg & Spa,"9-11 Rue de Pfaffenheim, Rouffach",4.4
8,Burgundy,France,47.0525,4.38372,Château de Gilly,"2 Place du Château, Gilly-lès-Cîteaux",4.3
9,Meursault,France,46.9782,4.76875,Château de Bellecroix,"20 Chemin de Bellecroix, Chagny",4.2
